In [1]:
#import libs
print("Importing Libs ...")
import numpy as np
import cv2
import os
import sqlite3
import datetime
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Importing Libs ...


Using TensorFlow backend.


In [2]:
#defining functions
print('Defining funtions ...')

def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT * FROM CLASSES')
    rows = cursorObj.fetchall()
    for row in rows:
        print(row)
def images_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT * FROM IMAGES')
    rows = cursorObj.fetchall()
    for row in rows:
        print(row)       
def make_class(name):
    path = "./classes/{}".format(name)
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the class %s failed" % name)
    else:
        print ("Successfully created the Class %s" % name)
        db.execute("INSERT INTO CLASSES (CLASS_NAME,NO_OF_OBJECTS) \
      VALUES (?, 0)",(name,));

def add_image(frame,CLASS_NAME):
    
    cursor = db.execute("SELECT NO_OF_OBJECTS FROM CLASSES WHERE CLASS_NAME = ?",(CLASS_NAME,))
    for row in cursor:
        objs = row[0]
        print(objs)
    objs +=1
    db.execute("UPDATE CLASSES set NO_OF_OBJECTS = ? where CLASS_NAME = ?",(objs,CLASS_NAME)) 
    db.commit()
    img_name = "{}_{}.png".format(CLASS_NAME,objs)
    cwd = os.getcwd()
    path = os.path.join(cwd , 'classes\{}'.format(CLASS_NAME))
    print(os.path.join(path , img_name))
    cv2.imwrite(os.path.join(path , img_name), frame)
    
    img_path = os.path.join(path , img_name)
    addImage_to_db(img_name,CLASS_NAME,datetime.datetime.now())
    
def create_table_class():
    db.execute('''CREATE TABLE CLASSES
             (CLASS_NAME           TEXT    NOT NULL,
             NO_OF_OBJECTS            INT     NOT NULL);''')   
def drop_class(CLASS_NAME):
    db.execute("DROP TABLE ?",(CLASS_NAME,))    
def create_table_images():
    db.execute('''CREATE TABLE IMAGES
             (NAME           TEXT    NOT NULL,
              TAG           TEXT    NOT NULL,
             TIME            timestamp);''')
def addImage_to_db(name, CLASS_NAME, stamp):

    cursor = db.cursor()
    # insert image detail
    sqlite_insert_with_param = """INSERT INTO IMAGES
                      ('NAME', 'TAG', 'TIME') 
                      VALUES (?, ?, ?);"""

    data_tuple = (name, CLASS_NAME, stamp)
    cursor.execute(sqlite_insert_with_param, data_tuple)
    db.commit()
    print("IMAGE added successfully \n")   


Defining funtions ...


In [3]:
# Connecting with database
print('connecting with database...')
db = sqlite3.connect("data.db")
print('Database: ')
sql_fetch(db)

connecting with database...
Database: 
('metal', 90)
('plastic', 152)
('cardboard', 51)
('trash', 1)
('paper', 75)
('glass', 0)


In [4]:
# db.execute("DROP TABLE CLASSES")

In [5]:
# create_table_class()

In [6]:
# # sql_fetch(db)
# make_class('metal')
# make_class('plastic')
# make_class('cardboard')
# make_class('trash')
# make_class('paper')
#make_class('glass')

In [7]:
# # getting lables 
# train=ImageDataGenerator(horizontal_flip=True,
#                          vertical_flip=True,
#                          validation_split=0.1,
#                          rescale=1./255,
#                          shear_range = 0.1,
#                          zoom_range = 0.1,
#                          width_shift_range = 0.1,
#                          height_shift_range = 0.1,)

# test=ImageDataGenerator(rescale=1/255,
#                         validation_split=0.1)

# train_generator=train.flow_from_directory('dataset-resized',
#                                           target_size=(384,512),
#                                           batch_size=32,
#                                           class_mode='categorical',
#                                           subset='training')

# test_generator=test.flow_from_directory('dataset-resized',
#                                         target_size=(384,512),
#                                         batch_size=32,
#                                         class_mode='categorical',
#                                         subset='validation')

# labels = (train_generator.class_indices)
# print(labels)
# labels = dict((v,k) for k,v in labels.items())

In [8]:
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.1,zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)
test=ImageDataGenerator(rescale=1/255,validation_split=0.1)
train_generator=train.flow_from_directory('dataset',target_size=(300,300),batch_size=32,
                                          class_mode='categorical',subset='training')
test_generator=test.flow_from_directory('dataset',target_size=(300,300),batch_size=32,
                                        class_mode='categorical',subset='validation')
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print(labels)

Found 2041 images belonging to 5 classes.
Found 225 images belonging to 5 classes.
{0: 'cardboard', 1: 'metal', 2: 'paper', 3: 'plastic', 4: 'wrappers'}


In [9]:
# create_table_images()

In [10]:
# images_fetch(db)

In [11]:
#loading pre trained model
print('loading models ...')
model = load_model('trained_model.h5')

loading models ...


In [12]:
# getting lables 
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory('dataset',
                                          target_size=(384,512),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory('dataset',
                                        target_size=(384,512),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

labels = (train_generator.class_indices)
print(labels)
labels = dict((v,k) for k,v in labels.items())

Found 2041 images belonging to 5 classes.
Found 225 images belonging to 5 classes.
{'cardboard': 0, 'metal': 1, 'paper': 2, 'plastic': 3, 'wrappers': 4}


In [13]:
# history = model.fit_generator(train_generator,
#                               epochs=10,
#                               steps_per_epoch=111//32,
#                               validation_data=test_generator,
#                               validation_steps=11//32,
#                               workers = 4) 

In [14]:
# model.save('model2.h5')

In [15]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

while True:
    ret, frame = cam.read()
    cv2.imshow("test", frame)
    if not ret:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        frame = increase_brightness(frame, value=20)
        img_name = "img.png"
        cv2.imwrite(img_name, frame)
        img_path = 'img.png'
        img = image.load_img(img_path, target_size=(384, 512))
        img = image.img_to_array(img, dtype=np.uint8)
        img=np.array(img)/255.0
        
        p=model.predict(img[np.newaxis, ...])
        predicted_class = labels[np.argmax(p[0], axis=-1)]
        print("Classified:",predicted_class)
        print(p[0])
        add_image(frame,predicted_class)
#         print("{} written!".format(img_name))
        img_counter += 1
           
cam.release()

cv2.destroyAllWindows()

Classified: plastic
[0.15099944 0.06860907 0.02335398 0.75703746]
152
C:\Users\Shubham Gandhi\Documents\Bin_data\classes\plastic\plastic_153.png
IMAGE added successfully 

Escape hit, closing...
